In [10]:
print("HELLO?")

HELLO?


In [11]:
# 1. Install pyflamegpu from the official wheelhouse
%pip install --extra-index-url https://whl.flamegpu.com/whl/cuda120/ pyflamegpu

# 2. Set the CUDA_PATH so FLAME GPU can find the compiler
import os
os.environ['CUDA_PATH'] = '/usr/local/cuda'

# 3. Verify installation
import pyflamegpu
print(f"FLAME GPU 2 version: {pyflamegpu.__version__}")

!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://whl.flamegpu.com/whl/cuda120/
FLAME GPU 2 version: 2.0.0rc4+cuda120
Wed Jan  7 22:43:52 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P0             27W /   70W |     106MiB /  15360MiB |      0%      Default |
|                  

In [12]:
# 1. Initialize the model and agent correctly
model = pyflamegpu.ModelDescription("Boids_Model")
bird_agent = model.newAgent("bird_agent")

# 2. Define 3D vectors as individual Float variables 
# This matches the C++ code: FLAMEGPU->getVariable<float>("x")
bird_agent.newVariableFloat("x")
bird_agent.newVariableFloat("y")
bird_agent.newVariableFloat("z")

bird_agent.newVariableFloat("vx")
bird_agent.newVariableFloat("vy")
bird_agent.newVariableFloat("vz")

bird_agent.newVariableFloat("fx")
bird_agent.newVariableFloat("fy")
bird_agent.newVariableFloat("fz")

In [13]:
# 1. Access the environment from the existing model
env = model.Environment()

# 2. Define scalar constants with initial values
# Based on your findings, use newPropertyFloat
env.newPropertyFloat("separation_weight", 1.5)
env.newPropertyFloat("alignment_weight", 1.0)
env.newPropertyFloat("cohesion_weight", 1.0)
env.newPropertyFloat("perception_radius", 50.0)
env.newPropertyFloat("max_speed", 4.0)
env.newPropertyFloat("cell_size", 10.0)

# 3. Define 3D vector bounds
bounds_min_val = [-100.0, -100.0, -100.0]
bounds_max_val = [100.0, 100.0, 100.0]

# Use newPropertyArrayFloat for the bounds
env.newPropertyArrayFloat("bounds_min", bounds_min_val)
env.newPropertyArrayFloat("bounds_max", bounds_max_val)

# 4. Set up the Spatial 3D Message list
message = model.newMessageSpatial3D("LocationMessage")

# No need to add x, y, z - they are already there!
# But we should add velocity so neighbors can align with each other
message.newVariableFloat("vx")
message.newVariableFloat("vy")
message.newVariableFloat("vz")

# Optional: add an ID so agents can ignore their own messages
message.newVariableID("id")

# Set the spatial bounds and link the search radius
message.setMin(bounds_min_val[0], bounds_min_val[1], bounds_min_val[2])
message.setMax(bounds_max_val[0], bounds_max_val[1], bounds_max_val[2])
message.setRadius(50.0)

In [14]:
# 1. Define the CUDA C++ logic for the agent function
# Note: For Spatial3D messages, we use setXYZ() for the reserved x,y,z variables

output_message_function_code = r'''
FLAMEGPU_AGENT_FUNCTION(output_message, flamegpu::MessageNone, flamegpu::MessageSpatial3D) {
    // Spatial3D messages have reserved x, y, z variables. 
    // We set them individually using setVariable.
    FLAMEGPU->message_out.setVariable<float>("x", FLAMEGPU->getVariable<float>("x"));
    FLAMEGPU->message_out.setVariable<float>("y", FLAMEGPU->getVariable<float>("y"));
    FLAMEGPU->message_out.setVariable<float>("z", FLAMEGPU->getVariable<float>("z"));
    
    // Set the velocity variables so neighbors can read them for Alignment
    FLAMEGPU->message_out.setVariable<float>("vx", FLAMEGPU->getVariable<float>("vx"));
    FLAMEGPU->message_out.setVariable<float>("vy", FLAMEGPU->getVariable<float>("vy"));
    FLAMEGPU->message_out.setVariable<float>("vz", FLAMEGPU->getVariable<float>("vz"));
    
    return flamegpu::ALIVE;
}
'''

# 2. Create the function description THROUGH the agent object
# Use newRTCFunction (Runtime Compiled Function) for C++ strings in Python
output_func = bird_agent.newRTCFunction("output_message", output_message_function_code)

# 3. Attach the message output to this specific function
output_func.setMessageOutput("LocationMessage")

In [ ]:
# 1. The Separation Kernel (CUDA C++)
# Note: Use getProperty for environment and msg.getXYZ() for spatial messages
separation_function_code = r'''
FLAMEGPU_AGENT_FUNCTION(separation, flamegpu::MessageSpatial3D, flamegpu::MessageNone) {
    const float3 my_pos = { FLAMEGPU->getVariable<float>("x"), 
                            FLAMEGPU->getVariable<float>("y"), 
                            FLAMEGPU->getVariable<float>("z") };
                            
    const float separation_radius = FLAMEGPU->environment.getProperty<float>("perception_radius");
    const float separation_weight = FLAMEGPU->environment.getProperty<float>("separation_weight");

    float3 steer = {0.0f, 0.0f, 0.0f};
    int count = 0;

    // Iterate over messages using the Spatial3D radius search
    for (const auto &msg : FLAMEGPU->message_in(my_pos.x, my_pos.y, my_pos.z)) {
        float3 other_pos = { msg.getX(), msg.getY(), msg.getZ() };
        
        // Calculate distance manually since we are using float3
        float dx = my_pos.x - other_pos.x;
        float dy = my_pos.y - other_pos.y;
        float dz = my_pos.z - other_pos.z;
        float dist_sq = dx*dx + dy*dy + dz*dz;

        if (dist_sq > 0 && dist_sq < (separation_radius * separation_radius)) {
            float dist = sqrtf(dist_sq);
            steer.x += (my_pos.x - other_pos.x) / dist;
            steer.y += (my_pos.y - other_pos.y) / dist;
            steer.z += (my_pos.z - other_pos.z) / dist;
            count++;
        }
    }

    if (count > 0) {
        // Simple normalization for steer
        float steer_mag = sqrtf(steer.x*steer.x + steer.y*steer.y + steer.z*steer.z);
        if (steer_mag > 0) {
            steer.x = (steer.x / steer_mag) * separation_weight;
            steer.y = (steer.y / steer_mag) * separation_weight;
            steer.z = (steer.z / steer_mag) * separation_weight;
        }
    }

    // Set the force variables we defined earlier
    FLAMEGPU->setVariable<float>("fx", steer.x);
    FLAMEGPU->setVariable<float>("fy", steer.y);
    FLAMEGPU->setVariable<float>("fz", steer.z);
    
    return flamegpu::ALIVE;
}
'''

# 2. Add function to the agent (already created in model)
# Use the bird_agent object you defined previously
separation_func = bird_agent.newRTCFunction("separation", separation_function_code)
separation_func.setMessageInput("LocationMessage")


In [16]:
# The alignment and cohesion kernels
alignment_function_code = r'''
FLAMEGPU_AGENT_FUNCTION(alignment, flamegpu::MessageSpatial3D, flamegpu::MessageNone) {
    const float3 my_pos = { FLAMEGPU->getVariable<float>("x"), 
                            FLAMEGPU->getVariable<float>("y"), 
                            FLAMEGPU->getVariable<float>("z") };
    const float perception_radius = FLAMEGPU->environment.getProperty<float>("perception_radius");
    const float alignment_weight = FLAMEGPU->environment.getProperty<float>("alignment_weight");

    float3 avg_vel = {0.0f, 0.0f, 0.0f};
    int count = 0;

    for (auto &msg : FLAMEGPU->message_in(my_pos.x, my_pos.y, my_pos.z)) {
        float3 other_pos = { msg.getX(), msg.getY(), msg.getZ() };
        float dx = my_pos.x - other_pos.x;
        float dy = my_pos.y - other_pos.y;
        float dz = my_pos.z - other_pos.z;
        float dist_sq = dx*dx + dy*dy + dz*dz;

        if (dist_sq > 0 && dist_sq < (perception_radius * perception_radius)) {
            avg_vel.x += msg.getVariable<float>("vx");
            avg_vel.y += msg.getVariable<float>("vy");
            avg_vel.z += msg.getVariable<float>("vz");
            count++;
        }
    }

    if (count > 0) {
        FLAMEGPU->setVariable<float>("fx", FLAMEGPU->getVariable<float>("fx") + (avg_vel.x / count * alignment_weight));
        FLAMEGPU->setVariable<float>("fy", FLAMEGPU->getVariable<float>("fy") + (avg_vel.y / count * alignment_weight));
        FLAMEGPU->setVariable<float>("fz", FLAMEGPU->getVariable<float>("fz") + (avg_vel.z / count * alignment_weight));
    }
    return flamegpu::ALIVE;
}
'''

cohesion_function_code = r'''
FLAMEGPU_AGENT_FUNCTION(cohesion, flamegpu::MessageSpatial3D, flamegpu::MessageNone) {
    const float3 my_pos = { FLAMEGPU->getVariable<float>("x"), 
                            FLAMEGPU->getVariable<float>("y"), 
                            FLAMEGPU->getVariable<float>("z") };
    const float perception_radius = FLAMEGPU->environment.getProperty<float>("perception_radius");
    const float cohesion_weight = FLAMEGPU->environment.getProperty<float>("cohesion_weight");

    float3 center_of_mass = {0.0f, 0.0f, 0.0f};
    int count = 0;

    for (auto &msg : FLAMEGPU->message_in(my_pos.x, my_pos.y, my_pos.z)) {
        float3 other_pos = { msg.getX(), msg.getY(), msg.getZ() };
        float dx = my_pos.x - other_pos.x;
        float dy = my_pos.y - other_pos.y;
        float dz = my_pos.z - other_pos.z;
        float dist_sq = dx*dx + dy*dy + dz*dz;

        if (dist_sq > 0 && dist_sq < (perception_radius * perception_radius)) {
            center_of_mass.x += other_pos.x;
            center_of_mass.y += other_pos.y;
            center_of_mass.z += other_pos.z;
            count++;
        }
    }

    if (count > 0) {
        center_of_mass.x /= (float)count;
        center_of_mass.y /= (float)count;
        center_of_mass.z /= (float)count;
        FLAMEGPU->setVariable<float>("fx", FLAMEGPU->getVariable<float>("fx") + (center_of_mass.x - my_pos.x) * cohesion_weight);
        FLAMEGPU->setVariable<float>("fy", FLAMEGPU->getVariable<float>("fy") + (center_of_mass.y - my_pos.y) * cohesion_weight);
        FLAMEGPU->setVariable<float>("fz", FLAMEGPU->getVariable<float>("fz") + (center_of_mass.z - my_pos.z) * cohesion_weight);
    }
    return flamegpu::ALIVE;
}
'''

integration_function_code = r'''
FLAMEGPU_AGENT_FUNCTION(integration, flamegpu::MessageNone, flamegpu::MessageNone) {
    float vx = FLAMEGPU->getVariable<float>("vx") + FLAMEGPU->getVariable<float>("fx");
    float vy = FLAMEGPU->getVariable<float>("vy") + FLAMEGPU->getVariable<float>("fy");
    float vz = FLAMEGPU->getVariable<float>("vz") + FLAMEGPU->getVariable<float>("fz");

    const float max_speed = FLAMEGPU->environment.getProperty<float>("max_speed");
    float speed = sqrtf(vx*vx + vy*vy + vz*vz);
    if (speed > max_speed) {
        vx = (vx / speed) * max_speed;
        vy = (vy / speed) * max_speed;
        vz = (vz / speed) * max_speed;
    }

    float next_x = FLAMEGPU->getVariable<float>("x") + vx;
    float next_y = FLAMEGPU->getVariable<float>("y") + vy;
    float next_z = FLAMEGPU->getVariable<float>("z") + vz;

    // Correct way to access environment array properties in FLAME GPU 2
    const float min_b = FLAMEGPU->environment.getProperty<float, 3>("bounds_min", 0);
    const float max_b = FLAMEGPU->environment.getProperty<float, 3>("bounds_max", 0);
    
    // Periodic Boundaries
    if (next_x < min_b) next_x = max_b; if (next_x > max_b) next_x = min_b;
    if (next_y < min_b) next_y = max_b; if (next_y > max_b) next_y = min_b;
    if (next_z < min_b) next_z = max_b; if (next_z > max_b) next_z = min_b;

    FLAMEGPU->setVariable<float>("x", next_x);
    FLAMEGPU->setVariable<float>("y", next_y);
    FLAMEGPU->setVariable<float>("z", next_z);
    FLAMEGPU->setVariable<float>("vx", vx);
    FLAMEGPU->setVariable<float>("vy", vy);
    FLAMEGPU->setVariable<float>("vz", vz);
    FLAMEGPU->setVariable<float>("fx", 0.0f);
    FLAMEGPU->setVariable<float>("fy", 0.0f);
    FLAMEGPU->setVariable<float>("fz", 0.0f);

    return flamegpu::ALIVE;
}
'''

alignment_func = bird_agent.newRTCFunction("alignment", alignment_function_code)
alignment_func.setMessageInput("LocationMessage")

cohesion_func = bird_agent.newRTCFunction("cohesion", cohesion_function_code)
cohesion_func.setMessageInput("LocationMessage")

integration_func = bird_agent.newRTCFunction("integration", integration_function_code)

In [17]:
# Layer 1: All agents write their position to the grid
lo = model.newLayer("L1_Output")
lo.addAgentFunction(output_func)

# Layer 2: Separation Force
ls = model.newLayer("L2_Separation")
ls.addAgentFunction(separation_func)

# Layer 3: Alignment Force
la = model.newLayer("L3_Alignment")
la.addAgentFunction(alignment_func)

# Layer 4: Cohesion Force
lc = model.newLayer("L4_Cohesion")
lc.addAgentFunction(cohesion_func)

# Layer 5: Movement (Integration)
lm = model.newLayer("L5_Movement")
lm.addAgentFunction(integration_func)

In [18]:
import random

# 1. Create the Simulation instance
sim = pyflamegpu.CUDASimulation(model)

# 2. Setup the Population
POPULATION_COUNT = 1000
pop = pyflamegpu.AgentVector(bird_agent, POPULATION_COUNT)

for i in range(POPULATION_COUNT):
    instance = pop[i]
    instance.setVariableFloat("x", random.uniform(-100, 100))
    instance.setVariableFloat("y", random.uniform(-100, 100))
    instance.setVariableFloat("z", random.uniform(-100, 100))
    instance.setVariableFloat("vx", random.uniform(-1, 1))
    instance.setVariableFloat("vy", random.uniform(-1, 1))
    instance.setVariableFloat("vz", random.uniform(-1, 1))
    instance.setVariableFloat("fx", 0.0)
    instance.setVariableFloat("fy", 0.0)
    instance.setVariableFloat("fz", 0.0)

# 3. Upload data to GPU and Run
sim.setPopulationData(pop)
print("Starting simulation...")
for step in range(100):
    sim.step()
print("Simulation complete!")

# 4. Retrieve data back to the Host
final_pop = pyflamegpu.AgentVector(bird_agent)
sim.getPopulationData(final_pop)

pos_x = [a.getVariableFloat("x") for a in final_pop]
pos_y = [a.getVariableFloat("y") for a in final_pop]
pos_z = [a.getVariableFloat("z") for a in final_pop]

FLAMEGPURuntimeException: (InvalidAgentFunc) /__w/FLAMEGPU2/FLAMEGPU2/src/flamegpu/detail/JitifyCache.cu(461): Error loading agent function (or function condition) ('cohesion'): function had compilation errors:
Compilation failed: NVRTC_ERROR_COMPILATION
cohesion_impl.cu(12): error: class "flamegpu::MessageSpatial3D::In::Filter::Message" has no member "getX"
          float3 other_pos = { msg.getX(), msg.getY(), msg.getZ() };
                                   ^

cohesion_impl.cu(12): error: class "flamegpu::MessageSpatial3D::In::Filter::Message" has no member "getY"
          float3 other_pos = { msg.getX(), msg.getY(), msg.getZ() };
                                               ^

cohesion_impl.cu(12): error: class "flamegpu::MessageSpatial3D::In::Filter::Message" has no member "getZ"
          float3 other_pos = { msg.getX(), msg.getY(), msg.getZ() };
                                                           ^
